In [1]:
Topics = {
"Streaming" : ["Quibi", "HBO", "Netflix", "Free Documentries", "Free Shows"],
"Exercises" : ["Yoga", "Walking", "Running", "Meditate", "Bicycle riding"],
"HBO_Shows" : ["Succession","True Blood","The Sopranos","Veep","The Wire"],
"Virtual_Games" : ["Animal Crossing: New Horizons", "Club Penguin", "Monopoly", "Mario Kart Tour","Words With Friends"],
"Cooking" : ["pancake", "pasta", "baking hacks", "bread", "cake"],
"Video_Chat_Platforms" : ["Google hangouts", "Zoom", "QuarantineChat", "HouseParty","Zoom Party"],
"Live_Virtual_Performance" : ["Billie Eilish","BTS", "Dua Lipa", "Sophia Ankel"],
"Cameo" : ["Steven Galanis", "Rachel Dratch", "Mandy Moore", "Restaurant Workers' Relief Fund"],
"Social_Media" : ["social media memes", "TikTok", "Instagram"],
"TikTok" : ["Hibachi grill","Hooters", "Ciara", "Usher", "Jennifer Lopez"],
"Instagram" : ["memes", "video chat", "makeup tutorials", "co-watching", "working_from_home"],
"Tiktok_Meme" : ["highschool musical", "dance workouts", "solo dance", "themed dinners", "distance dance"],
"Virtual_Workouts" : ["Gym", "Fat burning workout", "Dancing", "Toning workout", "HIIT workout"]
}

In [3]:
import pandas as pd
import pprint
from datetime import datetime, timedelta
from pytrends.request import TrendReq
import numpy as np

pytrends = TrendReq(hl='en-US', tz=360)
date_range = (datetime.today() - timedelta(150)).strftime('%Y-%m-%d') + " " + datetime.today().strftime('%Y-%m-%d')

In [4]:
def get_google_rel_queries(topics_dict):
    rel_queries_ls = []
    
    for i in topics_dict.keys():
        pytrends.build_payload(topics_dict[i], cat=0,timeframe = date_range , geo='US', gprop='')
        related_queries = pytrends.related_queries()
    
        for x in topics_dict[i]:
            try:
                j = ", ".join(list(related_queries[x]["top"]["query"][0:5]))
                rel_queries_ls.append({
                'query': x,
                'google_related_search': j
                }
                )
            except:
                continue
                
    g_df = pd.DataFrame(rel_queries_ls)
    return g_df

google_related_queries = get_google_rel_queries(Topics)

In [5]:
def interest_over_time(channels):
    
    pytrends.build_payload(channels, cat=0,timeframe = date_range , geo='US', gprop='')

    interest_over_time = pytrends.interest_over_time().drop("isPartial",axis = 1)
    
    return(interest_over_time)

In [6]:
ls = []

for i in Topics.keys(): 
    ls.append(interest_over_time(Topics[i]))
    
topic_df = pd.concat(ls, axis = 1)

In [7]:
topic_df = topic_df.T.reset_index()
topic_df.rename(columns = {'index': 'query'}, inplace = True)

In [8]:
topic_df = topic_df.merge(google_related_queries, how = "left", on="query")

In [9]:
import requests
from azure.cognitiveservices.search.websearch import WebSearchClient
from azure.cognitiveservices.search.websearch.models import SafeSearch
from msrest.authentication import CognitiveServicesCredentials

In [10]:
subscription_key = "865c1fcf4c99425c9892bddfe34d76ed"
assert subscription_key

In [11]:
def get_best_web_page(q):
    webpages = [] #For search result json get the query term, url, site name, and site_desc

    for v in search_results["webPages"]["value"]:
        webpages.append(
            {
            'query': q,
            'bing_url': v["url"],
            'bing_site_name': v["name"],
            'bing_site_desc':  v["snippet"]
            }
            )

    webpages_df = pd.DataFrame(webpages).iloc[0:1]
    return webpages_df

In [12]:
def get_related_queries(q):
    '''
    get's related queries for the search term and puts them into an array
    '''

    related_searches = []

    try:
        related_searches = search_results['relatedSearches']['value']

        for search in search_results['relatedSearches']['value']:
            related_searches.append(
                {
                    'query': q,
                    'bing_related_searches': search["text"],
                }
            )

    except KeyError:

        related_searches.append(
            {
                'query': q,
                'bing_related_searches': "",
            }
        )

    related_searches_df = pd.DataFrame(related_searches)

    related_searches_df = related_searches_df.groupby('query').apply(lambda x: ", ".join(x['bing_related_searches'].unique())).reset_index()

    related_searches_df.columns = ["query", "bing_related_searches"]
    return related_searches_df

In [13]:
dfObj = pd.DataFrame(columns=["query", "bing_url", "bing_site_name", "bing_site_desc", "bing_related_searches"])

In [14]:
for items in Topics.keys():
    for i in Topics[items]:

        headers = {"Ocp-Apim-Subscription-Key": subscription_key}
        params = {"q": i, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get("https://api.cognitive.microsoft.com/bing/v7.0/search", headers=headers, params=params)
        response.raise_for_status()
        search_results = response.json()

        merged_df = get_best_web_page(i).merge(get_related_queries(i), how='left', on='query')
        dfObj = dfObj.append(merged_df)

dfObj = dfObj.reset_index()
dfObj.drop(['index'], axis=1, inplace=True)

In [15]:
dfObj["bing_site_name"] = dfObj["bing_site_name"].str.replace('<[^<]+?>', '')
dfObj["bing_site_desc"] = dfObj["bing_site_desc"].str.replace('<[^<]+?>', '')
dfObj["bing_related_searches"] = dfObj["bing_related_searches"].str.replace('<[^<]+?>', '')

In [16]:
final_df = dfObj.merge(topic_df, how = 'left', on = 'query')

In [17]:
last_col = final_df.pop("google_related_search")
final_df.insert(5, "google_related_search", last_col)

In [19]:
from scipy import stats

axisvalues = list(range(1,len(final_df.iloc[:, 6:].columns)+1))

def calc_slope(row):
    a = scipy.stats.linregress(row, y = axisvalues)
    return a.slope 

calcs = final_df.iloc[:, 6:].apply(calc_slope,axis=1)

final_df['slope'] = calcs

C:\Users\I859302\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:130: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
C:\Users\I859302\Anaconda3\lib\site-packages\scipy\stats\_stats_mstats_common.py:142: RuntimeWarning: divide by zero encountered in double_scalars
  sterrest = np.sqrt((1 - r**2) * ssym / ssxm / df)


In [20]:
def trending(dataframe):
    if 0.9 < dataframe['slope'] < 1.1:
        return 'unchanged'
    elif dataframe['slope'] < 0.9:
        return 'decreasing'
    elif dataframe['slope'] > 1.1:
        return 'increasing'
    else:
        return 'NaN'

final_df['trending'] = final_df.apply(trending, axis=1)

In [21]:
final_df.to_excel("C:/Users/I859302/Desktop/trend_analysis.xlsx", sheet_name="trend_analysis", index=False)